In [1]:
import numpy as np

# 1- Résolution de Ax=b avec A une matrice carrée n×n et b∈Rn

In [2]:
A = np.random.randint(1, 10, (4, 4))
b = np.random.randint(1, 10, (4, ))


print(A, b)


[[9 4 2 2]
 [9 7 5 3]
 [9 7 8 1]
 [8 2 8 4]] [8 1 8 8]


## 1 - Résolution d'un système linéaire triangulaire

### Triangulaire Superieur

In [3]:
def solve_upper(A, b):
    assert A.shape[0] == b.shape[0]
    n = b.shape[0]
    x= np.zeros((n,))
    x[n-1] = b[n-1]/A[n-1, n-1]
    for i in range(n-2, -1, -1):
        x[i] = (b[i] - np.sum(A[i, i+1:]*x[i+1:]))/A[i, i]
    return x

Ai = np.random.randint(1, 10, (4, 4))
bi = np.random.randint(1, 10, (4, ))

mask_l = np.fromfunction(lambda i, j : i>j, (4,4))
Ai[mask_l] = 0
print(Ai, bi)
solve_upper(Ai, bi)

[[8 3 3 9]
 [0 5 9 9]
 [0 0 8 5]
 [0 0 0 7]] [8 2 9 1]


array([ 1.09642857, -1.72142857,  1.03571429,  0.14285714])

In [4]:
4/7

0.5714285714285714

### Triangulaire Inferieur

In [5]:
def solve_lower(A, b):
    assert A.shape[0] == b.shape[0]
    n = b.shape[0]
    x= np.zeros((n,))
    x[0] = b[0]/A[0, 0]
    for i in range(1, n):
        x[i] = (b[i] - np.sum(A[i, :i]*x[:i]))/A[i, i]
    return x

A1 = np.random.randint(1, 10, (4, 4))
b1 = np.random.randint(1, 10, (4, ))
mask_l = np.fromfunction(lambda i, j : i<j, (4,4))
A1[mask_l] = 0
print(A1, b1)
solve_lower(A1, b1)

[[1 0 0 0]
 [1 9 0 0]
 [4 4 5 0]
 [8 9 2 4]] [9 4 4 5]


array([  9.        ,  -0.55555556,  -5.95555556, -12.52222222])

## 2- Methode de Gauss

### Matrice Elementaire

In [6]:
def T(i, j, n):
    assert i < n and j < n
    A = np.eye(n)
    # i, j = i-1, j-1
    temp = A[i].copy()
    A[i] = A[j]
    A[j] = temp
    return A

def D(alpha, i,n):
    assert i < n
    # i = i-1
    A = np.eye(n)
    A[i, i] = alpha
    return A

def L(alpha, i, j, n):
    assert i < n and j < n
    A = np.eye(n)
    # i, j = i-1, j-1
    A [i, j] = alpha
    return (A)

def Mr(M, r , n=None, inv=False):
    if n is None:
        n = M.shape[0]
    assert 0 <= r < n , print(f"r : {r}, n : {n}")
    lambda_list = M[r+1:, r]
    A = np.eye(n)
    Ap = np.eye(n)
    for row in range(r+1, n):
        A = A@L(-lambda_list[row-r-1]/M[r, r], row, r, n)
        Ap = Ap@L(lambda_list[row-r-1]/M[r, r], row, r, n)

    return A, Ap

def Mr_up(M, r , n):
    assert r < n , print(f"r : {r}, n : {n}")
    lambda_list = M[:r, r]
    A = np.eye(n)
    Ap = np.eye(n)
    for row in range(r-1, -1, -1):
        A = A@L(-lambda_list[row]/M[r, r], row, r, n)
        Ap = Ap@L(lambda_list[row]/M[r, r], row, r, n)
    return A, Ap

In [7]:
def gauss(A, b):
    
    shape = A.shape
    n= shape[0]
    assert shape[0] == shape[1]
    assert shape[0] == b.size
    Ag = np.concatenate([A, b.reshape((n, 1))],axis=1)
    for i in range(0, shape[0]-1):
        j = np.argmax(np.abs(Ag[i+1:, i]))
        j += i + 1
        Ag = T(i, j, n)@Ag
        Ag = Mr(Ag,i, n)[0]@Ag
    return Ag[:, :n], Ag[:, n]

Ag, bg = gauss(A, b)

x = solve_upper(Ag, bg)
print("solution trouvé :", x)
print("verification : ", A@x)

solution trouvé : [ 2.24922118 -2.03115265  0.57632399 -2.63551402]
verification :  [8. 1. 8. 8.]


## 3- Résolution par décomposition LU

### 1- Décomposition LU de la matrice A

In [8]:
def Lu(A):
    n = A.shape[0]
    
    Upp = A.copy()
    Low = np.eye(n)

    for i in range(n-1):
        mr, mrinv = Mr(Upp, i, n)
        Upp = mr@Upp
        Low = Low@mrinv
    
    return Low, Upp

Low, Upp = Lu(A)

y = solve_lower(Low, b)
x = solve_upper(Upp, y)
print("solution trouvé :", x)
print("verification : ", A@x)

solution trouvé : [ 2.24922118 -2.03115265  0.57632399 -2.63551402]
verification :  [8. 1. 8. 8.]


In [9]:
A1 = np.array([[3, -2, 1], [2, 1, 1], [4, -3, 2]])
b1 = np.array([2, 7, 4])
ag, bg = gauss(A1, b1)
xg = solve_upper(ag, bg)
l, u = Lu(A1)
y = solve_lower(l, b1)
xlu = solve_upper(u, y)
print(xlu)
print(xg)
A1@xlu

[1. 2. 3.]
[1. 2. 3.]


array([2., 7., 4.])

## 4- Methodes iteratifs

### 1- Méthode de Jabobi

In [10]:
def jacobi(A, b, x0=None, maxiter=5, precision=10**-3):
    iters = 0
    xk = x0.copy()
    n = A.shape[0]
    print(x0)


    while np.linalg.norm(b-A@xk) > precision:
    #for i in range(5):
        xkp=xk.copy()
        for i in range(n): 
            xkm =xk.copy()
            xkm[i] = 0
            xkp[i] = (b[i] - A[i, :]@xkm)/A[i, i]
        xk = xkp

        iters += 1
        if iters == maxiter:
            break
        
        print(xk)
    return xk

jacobi(A, b, np.zeros((4,)))

[0. 0. 0. 0.]
[0.88888889 0.14285714 1.         2.        ]
[ 0.15873016 -2.57142857 -0.375      -1.84920635]
[2.52601411 0.99914966 3.30257937 3.71825397]
[ -1.11536281  -7.05739796  -3.18080357 -10.15676178]


array([ 6.98941362,  8.2017955 ,  9.6996016 , 14.12103175])

In [11]:
# methode vectorisée
def jacobi(A, b, x0=None, maxiter=5, precision=10**-3):
    i = 0
    xk = x0.copy()
    print(x0)
    while np.linalg.norm(b-A@xk) > precision:
        xkm = np.concatenate([[xk] for _ in range(4)], axis=0)*(1-np.eye(4))
        xkp = b - np.sum(A *xkm,axis= 1)
        xkp = xkp/np.diag(A)
        xk = xkp

        i+= 1
        if i == maxiter:
            break
        print(xk)
    
    return xk

jacobi(A, b, np.zeros((4,)))

[0. 0. 0. 0.]
[0.88888889 0.14285714 1.         2.        ]
[ 0.15873016 -2.57142857 -0.375      -1.84920635]
[2.52601411 0.99914966 3.30257937 3.71825397]
[ -1.11536281  -7.05739796  -3.18080357 -10.15676178]


array([ 6.98941362,  8.2017955 ,  9.6996016 , 14.12103175])

### 2- Méthode de Gauss-Seidel

In [12]:
def gauss_seidel(A, b, x0=None, maxiter=5, precision=10**-3):
    iters = 0
    xk = x0.copy()
    n = A.shape[0]
    print(x0)
    while np.linalg.norm(b-A@xk) > precision:
    #for i in range(5):
        xkp=xk.copy()
        for i in range(n): 
            xkm =xk.copy()
            xkm[i] = 0
            xkm[:i] = xkp[:i]
            # xkm[i+1:] = xk[i+1:]
            xkp[i] = (b[i] - A[i, :]@xkm)/A[i, i]
        xk = xkp

        print(xk)
        iters += 1
        if iters == maxiter:
            break
        
        
    return xk

gauss_seidel(A, b, np.zeros((4,)), )

[0. 0. 0. 0.]
[ 0.88888889 -1.          0.875      -1.02777778]
[ 1.36728395 -1.79960317  1.16493056 -2.16462743]
[ 1.91086738 -2.21836813  1.06192474 -2.83640018]
[ 2.26915816 -2.31754951  0.82960292 -3.03874739]
[ 2.40983189 -2.24575134  0.63381498 -2.96441806]


array([ 2.40983189, -2.24575134,  0.63381498, -2.96441806])

### 3- Méthode de la relaxation

In [13]:
def relaxation(A, b, x0=None, alpha=3**(-1), maxiter=5, precision=10**-3):
    assert alpha < 1
    iters = 0
    xk = x0.copy()
    n = A.shape[0]
    print(x0)
    while np.linalg.norm(b-A@xk) > precision:
    #for i in range(5):
        xkp=xk.copy()
        for i in range(n): 
            xkm =xk.copy()
            xkm[i] = 0
            xkm[:i] = xkp[:i]
            # xkm[i+1:] = xk[i+1:]
            xkp[i] = alpha* (b[i] - A[i, :]@xkm)/A[i, i] + (1-alpha)*xk[i]
        xk = xkp

        print(xk)
        iters += 1
        if iters == maxiter:
            break
        
        
    return xk

relaxation(A, b, np.zeros((4,)), maxiter=5)

[0. 0. 0. 0.]
[ 0.2962963  -0.07936508  0.24537037  0.31878307]
[ 0.46379581 -0.30802259  0.39954745  0.35496364]
[ 0.59523677 -0.55867025  0.52463985  0.24983639]
[ 0.71851816 -0.79336931  0.63463845  0.06334807]
[ 0.84114189 -1.00193732  0.73058964 -0.17193276]


array([ 0.84114189, -1.00193732,  0.73058964, -0.17193276])

# 2- Implantez la méthode du calcul de l'inverse d'une matrice carrée 𝐴 par "double triangulation", puis division des lignes par les éléments diagonaux.

In [69]:
def get_inv(A):
    n = A.shape[0]
    Ag = A.copy()
    M = np.eye(n)
    for i in range(0, n-1):
        M_down = Mr(Ag,i, n)[0]
        Ag = M_down@Ag 
        M = M_down@M


    for i in range(n-1, -1, -1):
        M_up = Mr_up(Ag,i, n)[0]
        Ag = M_up@Ag 
        M = M_up@M

    return np.linalg.inv(Ag)@M

def apply_zeros_mask(A):
    zeros_mask = lambda M: 10**(-14) < M < 10**(-14)
    A[zeros_mask(A)] = 0
    return A

get_inv(A)




array([[ 0.24610592, -0.19314642,  0.04984424,  0.00934579],
       [-0.15576324,  0.2741433 , -0.00623053, -0.12616822],
       [-0.11838006, -0.07165109,  0.1152648 ,  0.08411215],
       [-0.17757009,  0.39252336, -0.3271028 ,  0.12616822]])

In [66]:
np.linalg.inv(A)

array([[ 0.24610592, -0.19314642,  0.04984424,  0.00934579],
       [-0.15576324,  0.2741433 , -0.00623053, -0.12616822],
       [-0.11838006, -0.07165109,  0.1152648 ,  0.08411215],
       [-0.17757009,  0.39252336, -0.3271028 ,  0.12616822]])